## Introduction to Content Provenance

Content Credentials (C2PA) is a built-in feature of Amazon Nova Canvas that enables verification and tracking of AI-generated images. This capability adds cryptographically verifiable metadata to every generated image, documenting its origin, creation process, and any subsequent modifications. The metadata includes details such as generation timestamp, model information, and AWS's digital signature, enabling both automated and manual verification of authenticity.

### Use Case

OctankFashion wants to ensure transparency with their customers about their use of AI in their marketing materials. They need a reliable way to:
1. Prove the authenticity of their AI-generated product images
2. Track modifications made through their design workflow
3. Allow customers to verify the authenticity of marketing materials
4. Create a verification process for their creative team to ensure compliance

To demonstrate these capabilities, we'll explore how Content Credentials are:
- Automatically embedded during image generation
- Preserved and updated through editing operations
- Verified using both programmatic tools and user-friendly interfaces


<div class="alert alert-block alert-warning">
<b>Prerequisites:</b> Please run the prerequisites <b>00-prerequisites.ipynb</b> first before proceeding.
</div>


In [ ]:
import json
import base64
import boto3
from botocore.config import Config
from PIL import Image
from utils import (
    plot_images,
    save_binary_image,
    display_image_with_metadata,
    verify_c2pa_metadata,
    track_edit_provenance
)

bedrock_runtime_client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    config=Config(
        read_timeout=5 * 60
    ),
)
image_generation_model_id = "amazon.nova-canvas-v1:0"
output_dir = "output"

#### Example 1: Generating Images with Provenance

Amazon Nova Canvas automatically embeds Content Credentials metadata in every generated image. This metadata provides a cryptographically verifiable record that includes:

- **Manifest Identifier**: A unique UUID for the manifest
- **Generator Information**: Details about Nova Canvas and AWS Bedrock
- **Creation Time**: Precise timestamp of generation
- **Digital Signature**: AWS's cryptographic signature
- **AI Generation Markers**: Explicit indication of AI generation

The C2PA manifest includes several key components:
- `claim_generator`: Identifies the software/system that created the image
- `claim_generator_info`: Detailed information about the generation process
- `assertions`: List of claims about the image, including creation time and AI attribution
- `signature_info`: Cryptographic signature details from AWS

This metadata can be verified in two ways:
1. Through the [Content Credentials Verify](https://contentcredentials.org/verify) website
2. Programmatically using the [C2PA library](https://opensource.contentauthenticity.org/docs/c2pa-python/) as demonstrated below

In [ ]:
# Generate a simple product image
text = "A white t-shirt with a modern geometric pattern, professional product photography"

body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {"text": text},
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "width": 1024,
        "height": 1024,
        "cfgScale": 6.5,
        "seed": 42,
        "quality": "premium"
    }
})

print("Generating image...")
response = bedrock_runtime_client.invoke_model(
    body=body,
    modelId=image_generation_model_id,
    accept="application/json",
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
base64_image = response_body.get("images")[0]

# Save the image preserving C2PA metadata
image_path = f"{output_dir}/08-provenance-original.png"
save_binary_image(base64_image, image_path)
print(f"\nImage saved to: {image_path}")

# Display image and metadata using utils.plot_images
c2pa_info = display_image_with_metadata(image_path)

### Example 2: Verifying Image Provenance

Verification of Content Credentials is crucial for establishing trust in AI-generated content. This example demonstrates how to programmatically verify an image's authenticity using the C2PA Python library. The verification process includes:

**Verification Components:**
- **Digital Signature Validation**: Verifies AWS's cryptographic signature
- **Manifest Integrity**: Ensures the manifest hasn't been tampered with
- **Creation Information**: Validates timestamp and generator details
- **AI Attribution**: Confirms AI generation markers

**Key Metadata Fields:**
- `signature_info`: Contains the AWS digital signature and certificate information
- `actions`: Documents creation time and AI generation markers
- `claim_generator_info`: Provides details about the generation software

The verification can be integrated into automated workflows where programmatic verification is needed, while the Content Credentials Verify website provides a user-friendly interface for manual verification.


In [ ]:
# Verify an image with C2PA metadata
print("Verifying image provenance...")
verification_result = verify_c2pa_metadata(f"{output_dir}/08-provenance-original.png")

if verification_result['verified']:
    print("\nVerification Status: ✓ Image verified")
    print("\nSource Information:")
    print(f"Generator: {verification_result['source']['generator']}")
    print("\nGenerator Details:")
    for info in verification_result['source']['generator_info']:
        print(f"- {info['name']}" + (f" v{info['version']}" if 'version' in info else ""))

    print("\nCreation Information:")
    print(f"Action: {verification_result['creation']['action']}")
    print(f"Time: {verification_result['creation']['time']}")
    print(f"AI Generated: {'Yes' if verification_result['creation']['ai_generated'] else 'No'}")

    print("\nSignature Information:")
    sig_info = verification_result['signature']
    print(f"Issuer: {sig_info['issuer']}")
    print(f"Algorithm: {sig_info['alg']}")
    print(f"Time: {sig_info['time']}")
    print(f"Certificate SN: {sig_info['cert_serial_number']}")

    # Display the verified image
    img = Image.open(f"{output_dir}/08-provenance-original.png")
    plot_images([img], processed_title="Verified Image with C2PA Metadata")
else:
    print("\nVerification Status: ✗ Image not verified")
    print(f"Reason: {verification_result['reason']}")

### Example 3: Maintaining Provenance Through Edits

When editing images using Nova Canvas features like inpainting or outpainting, the Content Credentials are updated to reflect these modifications. Each edit operation generates a new manifest with updated metadata that documents the change while maintaining cryptographic verifiability.

**Key Changes in Edit Manifests:**
- **New Manifest ID**: A unique UUID for the edited version
- **Updated Action**: Changes from `c2pa.created` to `c2pa.edited`
- **New Timestamp**: Records when the edit occurred
- **Preserved Generator Info**: Maintains the link to Nova Canvas as the editing tool
- **Updated Signature**: New cryptographic signature validating the edit

**Key Fields for Edit Tracking:**
- `active_manifest`: Points to the most recent manifest
- `claim_generator`: Identifies Nova Canvas as the editing tool
- `actions`: Documents the edit operation with:
  - Type: `c2pa.edited`
  - Timestamp: When the edit occurred
  - Software agent: Nova Canvas
  - Digital source type: Confirms AI-based modification

This example demonstrates how Nova Canvas maintains verifiable provenance through editing operations by creating new signed manifests that document each modification.

In [ ]:
# First, let's read our original image
with open(f"{output_dir}/08-provenance-original.png", "rb") as image_file:
    original_image = base64.b64encode(image_file.read()).decode('utf8')

# Perform an inpainting operation to add a logo
body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": "Add a small company logo to the t-shirt",
        "image": original_image,
        "maskPrompt": "center of the t-shirt",
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "cfgScale": 6.5,
        "seed": 42,
        "quality": "premium"
    }
})

print("Performing inpainting...")
response = bedrock_runtime_client.invoke_model(
    body=body,
    modelId=image_generation_model_id,
    accept="application/json",
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
base64_image = response_body.get("images")[0]

# Save the edited image preserving C2PA metadata
edited_path = f"{output_dir}/08-provenance-edited.png"
save_binary_image(base64_image, edited_path)
print(f"\nEdited image saved to: {edited_path}")

# Compare provenance before and after editing
print("\nComparing manifests before and after editing...")
comparison = track_edit_provenance(
    f"{output_dir}/08-provenance-original.png",
    edited_path
)

# Display key changes summary
print("\nKey Changes Summary:")
print("-" * 50)
if 'error' not in comparison:
    original_time = comparison['original']['creation_time']
    edited_time = comparison['edited']['creation_time']
    print(f"Original Manifest ID: {comparison['original']['manifest_id']}")
    print(f"Created: {original_time}")
    print(f"\nEdited Manifest ID: {comparison['edited']['manifest_id']}")
    print(f"Modified: {edited_time}")
else:
    print(f"Error: {comparison['error']}")

## Take Away

Content provenance is a crucial feature of Amazon Nova Canvas that enables transparency and trust in AI-generated images. Through this notebook, we've explored how to:

1. Generate images with built-in provenance metadata
2. Verify the authenticity of Nova Canvas-generated images
3. Maintain provenance through editing operations

This capability is particularly valuable for businesses like OctankFashion that want to be transparent about their use of AI in marketing materials while maintaining trust with their customers.

Key points to remember:
- All Nova Canvas-generated images include provenance metadata by default
- Provenance information persists through supported editing operations
- Verification can be automated and integrated into existing workflows
